<span style="color:black;font-size:30pt"> Mitochondria assembly and species identification


<span style="color:blue;font-size:14pt"> We start by assembling the raw nanopore reads. We use genomeSize=16k as it is the expected size of a mitochondrial genome. </span>

In [4]:
 ~/canu/Linux-amd64/bin/canu -d assemble/ -p shark genomeSize=16k minReadLength=500 -nanopore-raw all_shark.fastq


-- Canu snapshot v1.7 +373 changes (r9065 568e52e019308accfd9727269085fe64cb527581)
--
-- CITATIONS
--
-- Koren S, Walenz BP, Berlin K, Miller JR, Phillippy AM.
-- Canu: scalable and accurate long-read assembly via adaptive k-mer weighting and repeat separation.
-- Genome Res. 2017 May;27(5):722-736.
-- http://doi.org/10.1101/gr.215087.116
-- 
-- Read and contig alignments during correction, consensus and GFA building use:
--   Šošic M, Šikic M.
--   Edlib: a C/C ++ library for fast, exact sequence alignment using edit distance.
--   Bioinformatics. 2017 May 1;33(9):1394-1395.
--   http://doi.org/10.1093/bioinformatics/btw753
-- 
-- Overlaps are generated using:
--   Berlin K, et al.
--   Assembling large genomes with single-molecule sequencing and locality-sensitive hashing.
--   Nat Biotechnol. 2015 Jun;33(6):623-30.
--   http://doi.org/10.1038/nbt.3238
-- 
--   Myers EW, et al.
--   A Whole-Genome Assembly of Drosophila.
--   Science. 2000 Mar 24;287(5461):2196-204.
--   http://doi.

--       2-     2  18228584 ********************************************************************** 0.9652 0.8400
--       3-     4   4594021 *****************                                                      0.9848 0.8862
--       5-     7    937974 ***                                                                    0.9937 0.9163
--       8-    11    321481 *                                                                      0.9969 0.9330
--      12-    16    147175                                                                        0.9982 0.9440
--      17-    22     76092                                                                        0.9989 0.9519
--      23-    29     42673                                                                        0.9993 0.9579
--      30-    37     26353                                                                        0.9995 0.9624
--      38-    46     16902                                                                     


-- Finished on Thu Aug 30 22:20:51 2018 (11 seconds) with 10.542 GB free disk space
----------------------------------------
-- Overlap store sorter finished.
----------------------------------------
-- Starting command on Thu Aug 30 22:20:51 2018 with 10.542 GB free disk space

    cd correction
    /home/ubuntu/canu/Linux-amd64/bin/ovStoreIndexer \
      -O  ./sark.ovlStore.BUILDING \
      -S ../sark.seqStore \
      -C  ./sark.ovlStore.config \
      -delete \
    > ./sark.ovlStore.BUILDING.index.err 2>&1

-- Finished on Thu Aug 30 22:20:51 2018 (lickety-split) with 11.053 GB free disk space
----------------------------------------
FROM: 'correction/sark.ovlStore.BUILDING'
TO:   'correction/sark.ovlStore'
-- Checking store.
----------------------------------------
-- Starting command on Thu Aug 30 22:20:51 2018 with 11.053 GB free disk space

    cd correction
    /home/ubuntu/canu/Linux-amd64/bin/ovStoreDump \
     -S ../sark.seqStore \
     -O  ./sark.ovlStore \
     -counts \
 

--    18000  18999      3 
--    19000  19999      1 
--    20000  20999      0 
--    21000  21999      1 
--    22000  22999      1 
--    23000  23999      3 
--    24000  24999      2 
--    25000  25999      1 
--    26000  26999      1 
--    27000  27999      1 
--    28000  28999      1 
--    29000  29999      1 
--    30000  30999      1 
--    31000  31999      1 
--    32000  32999      0 
--    33000  33999      1 
--    34000  34999      0 
--    35000  35999      0 
--    36000  36999      0 
--    37000  37999      0 
--    38000  38999      0 
--    39000  39999      0 
--    40000  40999      0 
--    41000  41999      0 
--    42000  42999      0 
--    43000  43999      0 
--    44000  44999      0 
--    45000  45999      0 
--    46000  46999      0 
--    47000  47999      0 
--    48000  48999      0 
--    49000  49999      0 
--    50000  50999      0 
--    51000  51999      1 
--
-- Purging correctReads output after loading into stores.
-- Purged 13 .cns out

----------------------------------------
--
-- Configured 1 overlapInCore jobs.
--
-- Running jobs.  First attempt out of 2.
----------------------------------------
-- Starting 'obtovl' concurrent execution on Thu Aug 30 22:22:45 2018 with 12.982 GB free disk space (1 processes; 1 concurrently)

    cd trimming/1-overlapper
    ./overlap.sh 1 > ./overlap.000001.out 2>&1

-- Finished on Thu Aug 30 22:30:45 2018 (480 seconds) with 12.98 GB free disk space
----------------------------------------
-- Found 1 overlapInCore output files.
--
-- overlapInCore compute 'trimming/1-overlapper':
--   kmer hits
--     with no overlap           548287      548287 +- 0
--     with an overlap            98431       98431 +- 0
--
--   overlaps                     98431       98431 +- 0
--     contained                      0           0 +- 0
--     dovetail                       0           0 +- 0
--
--   overlaps rejected
--     multiple per pair              0           0 +- 0
--     bad short windo

----------------------------------------
--
-- In sequence store './sark.seqStore':
--   Found 1700 reads.
--   Found 2377813 bases (148.61 times coverage).
--
--   Read length histogram (one '*' equals 15.07 reads):
--        0    999   1055 **********************************************************************
--     1000   1999    459 ******************************
--     2000   2999     84 *****
--     3000   3999     22 *
--     4000   4999     13 
--     5000   5999     11 
--     6000   6999      9 
--     7000   7999      6 
--     8000   8999      7 
--     9000   9999      1 
--    10000  10999      7 
--    11000  11999      2 
--    12000  12999      4 
--    13000  13999      4 
--    14000  14999      3 
--    15000  15999      4 
--    16000  16999      9 
--
-- Purging overlaps used for trimming.
----------------------------------------
-- Starting command on Thu Aug 30 22:30:47 2018 with 12.979 GB free disk space

    cd .
    /home/ubuntu/canu/Linux-amd64/bin/sqStoreD


    cd unitigging/1-overlapper
    ./overlap.sh 1 > ./overlap.000001.out 2>&1

-- Finished on Thu Aug 30 22:31:47 2018 (59 seconds) with 12.976 GB free disk space
----------------------------------------
-- Found 1 overlapInCore output files.
--
-- overlapInCore compute 'unitigging/1-overlapper':
--   kmer hits
--     with no overlap           114204      114204 +- 0
--     with an overlap           151058      151058 +- 0
--
--   overlaps                    151058      151058 +- 0
--     contained                  79552       79552 +- 0
--     dovetail                   71506       71506 +- 0
--
--   overlaps rejected
--     multiple per pair              0           0 +- 0
--     bad short window               0           0 +- 0
--     bad long window                0           0 +- 0
----------------------------------------
-- Starting command on Thu Aug 30 22:31:47 2018 with 12.976 GB free disk space

    cd unitigging
    /home/ubuntu/canu/Linux-amd64/bin/ovStoreConfig \
     -S 

--
-- Running jobs.  First attempt out of 2.
----------------------------------------
-- Starting 'oea' concurrent execution on Thu Aug 30 22:31:55 2018 with 12.966 GB free disk space (1 processes; 8 concurrently)

    cd unitigging/3-overlapErrorAdjustment
    ./oea.sh 1 > ./oea.000001.out 2>&1

-- Finished on Thu Aug 30 22:32:08 2018 (13 seconds) with 12.966 GB free disk space
----------------------------------------
-- Found 1 overlap error adjustment output files.
----------------------------------------
-- Starting command on Thu Aug 30 22:32:08 2018 with 12.966 GB free disk space

    cd unitigging/3-overlapErrorAdjustment
    /home/ubuntu/canu/Linux-amd64/bin/loadErates \
      -S ../../sark.seqStore \
      -O ../sark.ovlStore \
      -L ./oea.files \
    > ./oea.apply.err 2>&1

-- Finished on Thu Aug 30 22:32:08 2018 (like a bat out of hell) with 12.965 GB free disk space
----------------------------------------
-- No report available.
--
-- Running jobs.  First attempt out of

--
-- Contig sizes based on genome size 16kbp:
--
--            NG (bp)  LG (contigs)    sum (bp)
--         ----------  ------------  ----------
--     10       26939             1       26939
--     20       26939             1       26939
--     30       26939             1       26939
--     40       26939             1       26939
--     50       26939             1       26939
--     60       26939             1       26939
--     70       26939             1       26939
--     80       26939             1       26939
--     90       26939             1       26939
--    100       26939             1       26939
--    110       26939             1       26939
--    120       26939             1       26939
--    130       26939             1       26939
--    140       26939             1       26939
--    150       26939             1       26939
--    160       26939             1       26939
--    170       22380             2       49319
--    180       22380             2   

<span style="color:blue;font-size:14pt">we then blast the contigs agains nt

In [19]:
export BLASTDB=/home/ubuntu/DB
blastn -db nt -query shark.contigs.fasta -out contigs_nt.blastn -outfmt "6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -num_threads 16 -culling_limit 1 -evalue 0.001


In [33]:
cat contigs_nt.blastn

tig00000001	gi|19067985|gb|AY049832.1|	93.27	4025	0.0	Galeocerdo cuvier internal transcribed spacer 2, and 28S ribosomal RNA gene, partial sequence	AY049832	3987	9	258	4191
tig00000001	gi|19067986|gb|AY049833.1|	92.22	1787	0.0	Galeocerdo cuvier 18S ribosomal RNA gene, complete sequence	AY049833	1780	1	9695	11431
tig00000001	gi|501418415|gb|KC686343.1|	86.01	2358	0.0	Oncorhynchus mykiss clone 450D22 zinc finger CCHC domain-containing protein 3 pseudogene, complete sequence	KC686343	9561	11890	1628	3915
tig00000001	gi|501418415|gb|KC686343.1|	88.33	1354	0.0	Oncorhynchus mykiss clone 450D22 zinc finger CCHC domain-containing protein 3 pseudogene, complete sequence	KC686343	7951	9293	214	1523
tig00000001	gi|1220469783|gb|CP020797.1|	89.89	1602	0.0	Oryzias latipes strain HNI chromosome 19	CP020797	954244	955840	9671	11218
tig00000001	gi|22128507|gb|AF513986.1|	92.54	1300	0.0	Carcharhinus falciformis 5.8S ribosomal RNA gene, partial sequence; internal transcribed spacer 2, complete sequence;

tig00000756	gi|115334359|dbj|AB254130.1|	87.10	279	5e-79	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5980	5706	43	315
tig00000756	gi|115334359|dbj|AB254130.1|	86.40	272	5e-74	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5973	5706	647	915
tig00000756	gi|115334359|dbj|AB254130.1|	86.30	270	2e-72	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5973	5706	498	760
tig00000756	gi|115334359|dbj|AB254130.1|	84.59	279	3e-66	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5980	5706	193	461
tig00000756	gi|115334359|dbj|AB254130.1|	88.59	184	2e-52	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5980	5797	999	1177
tig00000756	gi|301128880|emb|FQ032659.1|	98.31	59	3e-17	Scyliorhinus canicula Cluster_HOXB sequence	FQ032659	78981	79038	205	263
tig00000756	gi|301128880|emb|FQ032659.1|	98.31	59	3e-17	Scyliorhinus canicula Cluster_HOXB sequence	FQ032659	78981	79038	652	710
tig00000756	gi|301128880|emb|FQ032659

tig00000757	gi|301128880|emb|FQ032659.1|	86.44	59	2e-04	Scyliorhinus canicula Cluster_HOXB sequence	FQ032659	78981	79038	829	882
tig00000757	gi|1379072074|gb|MH015179.1|	96.61	59	2e-15	Scyliorhinus canicula Fam45a protein (Fam45a), Eif3a protein (Eif3a), and Nanos1A protein (Nanos1A) genes, complete cds	MH015179	1749	1807	123	179
tig00000758	gi|115334359|dbj|AB254130.1|	86.40	272	2e-73	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5973	5706	255	521
tig00000758	gi|115334359|dbj|AB254130.1|	86.03	272	1e-71	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5973	5706	104	370
tig00000758	gi|115334359|dbj|AB254130.1|	83.64	275	5e-60	Sphyrna lewini GRLN gene for preproghrelin, complete cds	AB254130	5980	5710	801	1064
tig00000758	gi|30578123|dbj|AB074142.1|	90.61	213	5e-70	Triakis scyllium IL-1 gene for interleukin-1beta, complete cds	AB074142	3476	3266	1069	1277
tig00000758	gi|350626506|gb|JN646104.1|	84.57	162	7e-34	Triaenodon obesus selenium binding prote

<span style="color:blue;font-size:14pt"> tig00000010 has a long hit to "Carcharhinus obscurus mitochondrion". let's extract it </span>

In [9]:
samtools faidx shark.contigs.fasta 'tig00000010' > tig00000010.fasta

<span style="color:blue;font-size:14pt"> And let's also download the Carcharhinus obscurus mitochondrion genome

In [21]:
curl 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nuccore&id=KC470543&retnode=text&rettype=fasta' > 'Carcharhinus_obscurus_mitochondrion.fasta' 2> /dev/null


<span style="color:blue;font-size:14pt">Using Gepard we construct a dotplot of tig00000010 against itself. Note that the start and end of the sequence is highly similar. This is common when you assemble a circular genome 

<img src="files/tig10dotplot.jpeg">


<span style="color:blue;font-size:14pt">We then align the contig agains itself to find the ends of the repeated regions

In [22]:
nucmer --maxmatch --nosimplify tig00000010.fasta tig00000010.fasta
show-coords -lrcTH out.delta


1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 22175
# construct suffix tree for sequence of length 22175
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# CONSTRUCTIONTIME /usr/bin/mummer out.ntref 0.00
# reading input file "/home/ubuntu/git-presentations/shark_mito/tig00000010.fasta" of length 22174
# matching query-file "/home/ubuntu/git-presentations/shark_mito/tig00000010.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /usr/bin/mummer out.ntref 0.01
# SPACE /usr/bin/mummer out.ntref 0.04
4: FINISHING DATA
1	22174	1	22174	22174	22174	100.00	22174	22174	100.00	100.00	tig00000010	tig00000010
1	5753	16477	22174	5753	5698	94.58	22174	22174	25.94	25.70	tig00000010	tig00000010
16477	22174	1	5753	5698	5753	94.58	22174	22174	25.70	25.94	tig00000010	tig00000010


<span style="color:blue;font-size:14pt">This shows that region 1-5753 is 94.58% similar to region 16477-22174. We use samtools to delete region 16477-22174

In [13]:
samtools faidx tig00000010.fasta tig00000010:1-16476 > tig00000010_cut.fasta

<span style="color:blue;font-size:14pt">The dotplot show no repeated regeions
<img src="files/tig10_cut.jpeg">

<span style="color:blue;font-size:14pt">Aligning agains the Carcharhinus obscurus mitocondrion genome shows that tig00000010 is shifted. By convention, mitochondrial genomes are reported as lineal genomes starting on the tRNA-phe gene. Canu has no way of finding this gene, so the starting point is mostly random. 


<img src="files/tig10_vs_obscurus.jpeg">

<span style="color:blue;font-size:14pt">We aling Carcharhinus obscurus mitochondrion genome to tig00000010 to find the breaking point. In this case, it is between positions 8309 and 8310

In [15]:
nucmer -p cut_vs_carcharhinus tig00000010_cut.fasta Carcharhinus_obscurus_mitochondrion.fasta
show-coords -lrcTH cut_vs_carcharhinus.delta

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "cut_vs_carcharhinus.ntref" of length 16477
# construct suffix tree for sequence of length 16477
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# CONSTRUCTIONTIME /usr/bin/mummer cut_vs_carcharhinus.ntref 0.00
# reading input file "/home/ubuntu/git-presentations/shark_mito/Carcharhinus_obscurus_mitochondrion.fasta" of length 16706
# matching query-file "/home/ubuntu/git-presentations/shark_mito/Carcharhinus_obscurus_mitochondrion.fasta"
# against subject-file "cut_vs_carcharhinus.ntref"
# COMPLETETIME /usr/bin/mummer cut_vs_carcharhinus.ntref 0.01
# SPACE /usr/bin/mummer cut_vs_carcharhinus.ntref 0.03
4: FINISHING DATA
1	8309	8239	16706	8309	8468	91.96	16476	16706	50.43	50.69	tig00000010:1-16476	KC470543.1
8310	16476	1	8238	8167	8238	95.29	16476	16706	49.57	49.31	tig00000010:1-16476	KC470543.1


<span style="color:blue;font-size:14pt">We rotate the genome using samtools

In [24]:
samtools faidx tig00000010_cut.fasta 'tig00000010:1-16476':8310-16476 'tig00000010:1-16476':1-8309 | grep -v '>' | tr -d '\n' | sed '1s/^/>tig00000010_cut_rotated\n/' | sed -e '$a\'  > tig00000010_cut_rotated.fasta

<span style="color:blue;font-size:14pt">The dotplot show tig00000010 has the standar starting point now
<img src="files/cut_rotated_vs_obscurus.jpeg">

<span style="color:blue;font-size:14pt"> We use <a href="http://mitofish.aori.u-tokyo.ac.jp/annotation/input.html">MitoFish server</a> to anotate tig00000010. this gives us a <a href="files/tig00000010_cut_rotated_genes.fa">fasta file</a> with genes. We then blast them agains nt. 
 

In [31]:
blastn -db nt -query tig00000010_cut_rotated_genes.fa -out genes.blast -outfmt "6 qseqid sseqid pident length evalue sscinames salltitles sallacc sstart send qstart qend" -max_target_seqs 1 -num_threads 16 -culling_limit 1 -max_hsps 1 -evalue 0.001

<span style="color:blue;font-size:14pt">We count how many genes blast assings to each shark species

In [32]:
cut -f6 genes.blast | sort | uniq -c | sort -nr

     22 Carcharhinus falciformis
      9 Carcharhinus obscurus
      1 Sphyrna lewini
      1 Prionace glauca
      1 Mustelus widodoi
      1 Carcharhinus macloti
      1 Carcharhinus brevipinna
      1 Carcharhinus amblyrhynchos


<span style="color:blue;font-size:14pt">With this information we can conclude that the sample was Carcharhinus falciformis